In [5]:
# 시스템 업데이트 및 크롬 브라우저 설치
!apt-get update
!apt-get install -y fonts-nanum fonts-nanum-coding  # 한글 폰트 깨짐 방지 (선택)
!wget -q -O - https://dl-ssl.google.com/linux/linux_signing_key.pub | apt-key add -
!sh -c 'echo "deb [arch=amd64] http://dl.google.com/linux/chrome/deb/ stable main" >> /etc/apt/sources.list.d/google-chrome.list'
!apt-get update
!apt-get install -y google-chrome-stable

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://cli.github.com/packages stable/main amd64 Packages [356 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [85.0 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,728 kB]
Get:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,904 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/multiverse amd64 Packages [62.6 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Pac

In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

# 브라우저 옵션 설정
options = Options()
options.add_argument('--headless') # 중요: 서버 환경이라 화면을 띄울 수 없음
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# 드라이버 실행
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# 테스트 접속
driver.get("https://www.google.com")
print(driver.title)

# 작업 완료 후 닫기
# driver.quit()

Google


In [1]:
pip install selenium

In [3]:
pip install webdriver-manager

로컬아닌 코랩에서 크롤링하는 코드

In [7]:
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# 1. 드라이버 설정 (코랩 환경 최적화)
options = Options()
options.add_argument('--headless')          # 화면 없이 실행 (코랩 필수)
options.add_argument('--no-sandbox')        # 보안 제한 해제 (코랩 필수)
options.add_argument('--disable-dev-shm-usage') # 메모리 부족 방지
# options.add_argument('--disable-gpu')     # 필요 시 추가

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

all_data = []

try:
    driver.get("https://glycemicindex.com/gi-search/")
    wait = WebDriverWait(driver, 15) # 대기 시간을 15초로 넉넉히 설정

    for page in range(1, 429):
        print(f"현재 {page} 페이지 크롤링 중...")

        # 테이블의 데이터 행이 나타날 때까지 대기
        # 'tablepress-1' 안의 'tr'이 최소 1개 이상 나타날 때까지 기다립니다.
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#tablepress-1 tbody tr")))

        # 현재 페이지의 행들 가져오기
        rows = driver.find_elements(By.CSS_SELECTOR, "#tablepress-1 tbody tr")

        for row in rows:
            cols = row.find_elements(By.TAG_NAME, "td")
            # 데이터가 비어있는 행(No data available) 방지
            if len(cols) >= 7:
                data = {
                    "Food Name": cols[0].text,
                    "GI": cols[1].text,
                    "Category": cols[2].text,
                    "Country": cols[3].text,
                    "Serving Size": cols[4].text,
                    "Carbs per Serve": cols[5].text,
                    "GL": cols[6].text
                }
                all_data.append(data)

        # 다음 페이지 버튼 클릭
        if page < 428:
            try:
                # Next 버튼이 화면에 보이고 클릭 가능할 때까지 대기
                next_button = wait.until(EC.element_to_be_clickable((By.ID, "tablepress-1_next")))
                driver.execute_script("arguments[0].click();", next_button)

                # 페이지 번호가 바뀌었는지 확인하기 위한 짧은 대기 (중요)
                time.sleep(2)
            except Exception as e:
                print(f"{page}페이지에서 다음으로 넘어가지 못했습니다. (종료)")
                break

except Exception as main_e:
    print(f"오류 발생: {main_e}")

finally:
    driver.quit()
    print("드라이버 종료.")

# 2. 데이터 저장
if all_data:
    df = pd.DataFrame(all_data)
    df.to_csv("gi_database_full.csv", index=False, encoding="utf-8-sig")
    print(f"총 {len(df)}개의 데이터 저장 완료! (gi_database_full.csv)")
else:
    print("수집된 데이터가 없습니다.")

현재 1 페이지 크롤링 중...
현재 2 페이지 크롤링 중...
현재 3 페이지 크롤링 중...
현재 4 페이지 크롤링 중...
현재 5 페이지 크롤링 중...
현재 6 페이지 크롤링 중...
현재 7 페이지 크롤링 중...
현재 8 페이지 크롤링 중...
현재 9 페이지 크롤링 중...
현재 10 페이지 크롤링 중...
현재 11 페이지 크롤링 중...
현재 12 페이지 크롤링 중...
현재 13 페이지 크롤링 중...
현재 14 페이지 크롤링 중...
현재 15 페이지 크롤링 중...
현재 16 페이지 크롤링 중...
현재 17 페이지 크롤링 중...
현재 18 페이지 크롤링 중...
현재 19 페이지 크롤링 중...
현재 20 페이지 크롤링 중...
현재 21 페이지 크롤링 중...
현재 22 페이지 크롤링 중...
현재 23 페이지 크롤링 중...
현재 24 페이지 크롤링 중...
현재 25 페이지 크롤링 중...
현재 26 페이지 크롤링 중...
현재 27 페이지 크롤링 중...
현재 28 페이지 크롤링 중...
현재 29 페이지 크롤링 중...
현재 30 페이지 크롤링 중...
현재 31 페이지 크롤링 중...
현재 32 페이지 크롤링 중...
현재 33 페이지 크롤링 중...
현재 34 페이지 크롤링 중...
현재 35 페이지 크롤링 중...
현재 36 페이지 크롤링 중...
현재 37 페이지 크롤링 중...
현재 38 페이지 크롤링 중...
현재 39 페이지 크롤링 중...
현재 40 페이지 크롤링 중...
현재 41 페이지 크롤링 중...
현재 42 페이지 크롤링 중...
현재 43 페이지 크롤링 중...
현재 44 페이지 크롤링 중...
현재 45 페이지 크롤링 중...
현재 46 페이지 크롤링 중...
현재 47 페이지 크롤링 중...
현재 48 페이지 크롤링 중...
현재 49 페이지 크롤링 중...
현재 50 페이지 크롤링 중...
현재 51 페이지 크롤링 중...
현재 52 페이지 크롤링 중...
현재 53 페이지 크롤링 중...
현재